In [7]:
"""Retrieve classAds job information using Spark"""

from __future__ import print_function

from subprocess import Popen, PIPE, STDOUT
from pyspark.sql import Column
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
import pyspark.sql.types as types
import schemas

In [8]:
csvreader = spark.read.format("com.databricks.spark.csv").option("nullValue","null").option("mode", "FAILFAST")

# Path where the input files are
basepath="/project/awg/cms/CMS_DBS3_PROD_GLOBAL/current"

# Get the information about Blocks so that we can map the block name of the job to the block id 
dbs_datasets = csvreader.schema(schemas.schema_datasets()).load(basepath+"/DATASETS")

In [9]:
# dbs_datasets.printSchema()

In [10]:
data_tiers_list=["MINIAOD","MINIAODSIM","NANOAOD","NANOAODSIM","AOD","AODSIM","RAW","GEN-SIM","USER"]

inputfile="/project/monitoring/archive/condor/raw/metric/2019/06/02/*.json.gz"

jobreports = spark.read.json(inputfile)

In [11]:
# The following regexps describe what is in the cache
regexp1="/*/Run2016.*-03Feb2017.*/MINIAOD"
regexp2="/*/RunIISummer16MiniAODv2-PUMoriond17_80X_.*/MINIAODSIM"
regexp3="/*/.*-31Mar2018.*/MINIAOD"
regexp4="/*/.*RunIIFall17MiniAODv2.*/MINIAODSIM"

# Desired sites
sites = ["T2_US_UCSD", "T2_US_Caltech", "T3_US_UCR"]

ds = (jobreports
        # Joing dataset DBS table with jobreports
        .join(dbs_datasets, col('data.DESIRED_CMSDataset')==col('d_dataset'))
        # Require datasets from cache
        .filter(
                col('d_dataset').rlike(regexp1) |
                col('d_dataset').rlike(regexp2) | 
                col('d_dataset').rlike(regexp3) | 
                col('d_dataset').rlike(regexp4)
               )
        # Require at UCSD, Caltech, or UCR
        .filter(col('data.CMSSite').isin(sites))
        # Require CMS jobs
        .filter(col('data.VO') == "cms")
        # Require analysis jobs
        .filter(col('data.Type') == 'analysis')
        # Require completed jobs
        .filter(col('data.Status') == 'Completed')
        # There are other utility CRAB jobs we don't want to read 
        .filter(col('data.JobUniverse') == 5)
        # Select columns to save
        .select(
               col('data.CMSSite').alias('site'),
               col('data.DESIRED_CMSDataset').alias('dataset'),
               col('data.CRAB_Workflow').alias('workflow_id'),
               col('data.CRAB_Id').alias('crab_id'),
               col('data.JobStartDate').alias('start_date'),
               col('data.ScheddName').alias('schedd_name')
               )
    )

In [13]:
(ds.write
   .option("compression","gzip")
   .mode("overwrite")
   .parquet("hdfs://analytix/user/jguiang/shared/ClassAds_06-02-2019")
)

In [14]:
# Move from hdfs to eos
p = Popen("hdfs dfs -get /user/jguiang/shared/ClassAds_06-02-2019 /eos/user/j/jguiang/data-access/parquet/",
          shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)

print (p.stdout.read())